In [0]:
dbutils.widgets.text("year","")
dbutils.widgets.text("month","")
dbutils.widgets.text("day","")
dbutils.widgets.text("storageName","")

In [0]:
year = dbutils.widgets.get("year")
month = dbutils.widgets.get("month")
day = dbutils.widgets.get("day")
storageName = dbutils.widgets.get("storageName")

In [0]:
parts_df = spark.read.option("inferSchema","true").option("header","true").csv(f"abfss://raw@{storageName}.dfs.core.windows.net/files/parts/Year={year}/Month={month}/Day={day}/*.csv.gz")

In [0]:
parts_df.show()

+--------+--------------------+-----------+-------------+
|part_num|                name|part_cat_id|part_material|
+--------+--------------------+-----------+-------------+
|  003381|Sticker Sheet for...|         58|      Plastic|
|  003383|Sticker Sheet for...|         58|      Plastic|
|  003402|Sticker Sheet for...|         58|      Plastic|
|  003429|Sticker Sheet for...|         58|      Plastic|
|  003432|Sticker Sheet for...|         58|      Plastic|
|  003434|Sticker Sheet for...|         58|      Plastic|
|  003435|Sticker Sheet for...|         58|      Plastic|
|  003436|Sticker Sheet for...|         58|      Plastic|
|  003437|Sticker Sheet for...|         58|      Plastic|
|  003438|Sticker Sheet for...|         58|      Plastic|
|  003439|Sticker Sheet for...|         58|      Plastic|
|  003440|Sticker Sheet for...|         58|      Plastic|
|  003446|Sticker Sheet for...|         58|      Plastic|
|  003496|Sticker Sheet for...|         58|      Plastic|
|  003497|Stic

In [0]:
# Rename columns
parts_df = parts_df.withColumnRenamed("part_num","partID").withColumnRenamed("name","partName").withColumnRenamed("part_material","partMaterial")

In [0]:
# New data frame from another file
categories_df = spark.read.option("inferSchema","true").option("header","true").csv(f"abfss://raw@{storageName}.dfs.core.windows.net/files/part_categories/Year={year}/Month={month}/Day={day}/*.csv.gz")

In [0]:
categories_df.show()

+---+--------------------+
| id|                name|
+---+--------------------+
|  1|          Baseplates|
|  3|       Bricks Sloped|
|  4|Duplo, Quatro and...|
|  5|      Bricks Special|
|  6|       Bricks Wedged|
|  7|          Containers|
|  8|      Technic Bricks|
|  9|      Plates Special|
| 11|              Bricks|
| 12|  Technic Connectors|
| 13|            Minifigs|
| 14|              Plates|
| 15|       Tiles Special|
| 16|   Windows and Doors|
| 17|            Non-LEGO|
| 18|Hinges, Arms and ...|
| 19|               Tiles|
| 20|Bricks Round and ...|
| 21|Plates Round Curv...|
| 22|          Pneumatics|
+---+--------------------+
only showing top 20 rows



In [0]:
categories_df = categories_df.withColumnRenamed("name","categoryName")

In [0]:
# Merging both data frames
merged_df = parts_df.join(categories_df, parts_df.part_cat_id == categories_df.id, how='left').drop('id', 'part_cat_id')

In [0]:
merged_df.show()

+------+--------------------+------------+------------+
|partID|            partName|partMaterial|categoryName|
+------+--------------------+------------+------------+
|003381|Sticker Sheet for...|     Plastic|    Stickers|
|003383|Sticker Sheet for...|     Plastic|    Stickers|
|003402|Sticker Sheet for...|     Plastic|    Stickers|
|003429|Sticker Sheet for...|     Plastic|    Stickers|
|003432|Sticker Sheet for...|     Plastic|    Stickers|
|003434|Sticker Sheet for...|     Plastic|    Stickers|
|003435|Sticker Sheet for...|     Plastic|    Stickers|
|003436|Sticker Sheet for...|     Plastic|    Stickers|
|003437|Sticker Sheet for...|     Plastic|    Stickers|
|003438|Sticker Sheet for...|     Plastic|    Stickers|
|003439|Sticker Sheet for...|     Plastic|    Stickers|
|003440|Sticker Sheet for...|     Plastic|    Stickers|
|003446|Sticker Sheet for...|     Plastic|    Stickers|
|003496|Sticker Sheet for...|     Plastic|    Stickers|
|003497|Sticker Sheet for...|     Plastic|    St

In [0]:
merged_df_cleaned = merged_df.dropDuplicates()

In [0]:
merged_df_cleaned.write.format("delta").mode("overwrite").save(f"abfss://curated@{storageName}.dfs.core.windows.net/files/parts/")